数据准备与导入

In [76]:
import pandas as pd #数据表
import numpy as np    #数据处理
import re
import jieba   #中文分词
import matplotlib.pyplot as plt  #
from gensim import corpora, models   #gensim是一个自然语言处理的库
import pyLDAvis   
import pyLDAvis.gensim_models as gensimvis    #gensim的可视化库

In [77]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col=0)

语料预处理


In [78]:

# 文本清洗与分词函数
def clean_text(text):
    #剔除符号与数字，只保留汉字
    processed = re.sub('[^\u4e00-\u9fa5]+','',text)
    #分词
    words = jieba.lcut(processed)
    #剔除停用词
    stopwords = ['的','了','在','和','是','我','有',
                 '就','不','人','都','说','要','这','也','为','他',
                 '她','它','一个','上','去','会','着','对','也','吗']
    words = [w for w in words if w not in stopwords]
    #return words
    return ' '.join(words)

# 测试函数
#print(df['标题/微博内容'][0])
#print(clean_text(df['标题/微博内容'][0]))


In [79]:
df['微博内容分词'] = df['标题/微博内容'].astype(str).apply(clean_text)   # 对原始文本进行分词处理
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())    # 将分词结果转换为列表形式
#df['微博内容分词']


LDA主题模型

In [80]:
texts = [['human', 'interface', 'computer']]
# 创建词典
dictionary = corpora.Dictionary(texts)  
print(dictionary.token2id)
# convert tokenized document into bag-of-words (BoW) format
# format: list of (token_id, token_count) tuples
print(dictionary.doc2bow(['human', 'interface', 'computer']))



{'computer': 0, 'human': 1, 'interface': 2}
[(0, 1), (1, 1), (2, 1)]


In [81]:
dictionary.add_documents([["cat","say","meow"],["dog"]])   # 添加新文档
#print(dictionary.token2id)   # 查看词典
#print(dictionary.doc2bow(["dog","computer","non_existent_word"]))   # 查看新文档的BoW格式

In [82]:
dictionary = corpora.Dictionary(df['微博内容分词']) #根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']] #根据分词结果创建语料库
#dictionary

#corpus



In [83]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5,id2word=dictionary, passes=15)

In [84]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
#for topic in topics:
#    print(topic)

In [85]:
# 按主题权重降序排列并打印前5个关键词
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):   
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.9993657469749451	 Topic: 0.009*"月" + 0.006*"新" + 0.005*"数字" + 0.005*"从" + 0.004*"年"


In [86]:
def infer_topic(lda_model, document):   
    bow = dictionary.doc2bow(document)     # 将分词列表转换为词袋向量 (word_id, word_count)
    topics = lda_model.get_document_topics(bow)  # 获取文档的主题分布
    return topics


docunments = df['微博内容分词'].values.tolist()  # 获取所有文档的分词列表
for i, doc in enumerate(docunments):  # 遍历每个文档
    # 对每个文档进行主题推断        
    doc_topics = infer_topic(lda_model, doc)
    # 打印每个文档的主题分布
    #print(f"Document {i+1}:")
    #print(doc_topics)
    #print()
    
    

可视化

In [87]:
#如果在数据量较大时，使用pyLDAvis可视化LDA模型
lda_vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
# 可视化LDA模型
pyLDAvis.display(lda_vis_data)